## Project : Solar Panels Productivity
#### Source
https://www.kaggle.com/datasets/ibrahimkiziloklu/solar-radiation-dataset  
https://www.kaggle.com/competitions/2021-ai-w6-p2/data


Net class

https://tutorials.pytorch.kr/beginner/blitz/neural_networks_tutorial.html
reset weight and bias : https://discuss.pytorch.org/t/reset-model-weights/19180/3


In [80]:
import numpy

#get data from csv
data = numpy.genfromtxt("C:/Users/OWO/Documents/AA_CODE/@Projects/Projects/ai_0007_nn_Solar_Panles/train.csv",delimiter=",",encoding='UTF-8')


#make list of features
data_train = data[1:,:-1]
print("row : ",len(data_train))
print("col : ",len(data_train[0]))
#print(data_train)

#make list of outputs
data_y_original = data[1:,-1:]
#print(data_y)

#make a 2 dim list [data num][possible output num]
data_y = [[0 for col in range(int(data_y_original.max())+3)] for row in range(len(data_y_original))] 
print("data Y\nrow : ",len(data_y))
print("col : ",len(data_y[0]))
#print(data_y)

for row in range(len(data_y)):
  data_y[row][int(data_y_original[row][0])] = 1
  #print(int(data_y_original[row][0]))
#I could use round (반올림) but thats too compliacted so I didnt

row :  9999
col :  8
data Y
row :  9999
col :  100


## Hardwear

In [81]:
##Hardwere
import torch
if torch.cuda.is_available() == True:
  device = 'cuda'
  templist = [1,2,3]
  templist = torch.FloatTensor(templist).to(device)
  print("Cuda torch working : ",end="")
  print(templist.is_cuda)
  print("current device no. : ",end="")
  print(torch.cuda.current_device())
  print("GPU device count : ",end="")
  print(torch.cuda.device_count())
  print("GPU name : ",end="")
  print(torch.cuda.get_device_name(0))
  print("device : ",device)
elif torch.backends.mps.is_available() == True:
  print("Apple device detected\nActivating Apple Silicon GPU")
  device = torch.device("mps")
else:
  print("cant use gpu , activating cpu")
  device = 'cpu'

Cuda torch working : True
current device no. : 0
GPU device count : 1
GPU name : NVIDIA GeForce GTX 1080
device :  cuda


## ML

In [88]:
##ML2
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim


class Net(nn.Module):
    def __init__(self):
        super().__init__()
        drop_prob = 0.2 # 0.2 == 20%
        self.fc1 = nn.Linear(8, 16,bias=True)
        self.fc2 = nn.Linear(16, 32,bias=True)
        self.fc3 = nn.Linear(32, 64,bias=True)
        self.fc4 = nn.Linear(64, 100)
        self.relu = torch.nn.Sigmoid()
        self.dropout = torch.nn.Dropout(p=drop_prob)# nn.Dropout module is a regularization technique for reducing overfitting in neural networks by randomly setting a proportion of input units to 0 during training
        self.reset_weights()
        
        ## initialize weight

    def forward(self, x):
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        x = self.fc4(x)
        
        return x
    
    def reset_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight.data)
                if m.bias is not None:
                    m.bias.data.zero_()

net = Net()
net=net.to(device)




#lr_list = [10000,100,1,1e-3,1e-5,1e-10]
lr_list = [1e-3]

for LR in lr_list:
    net.reset_weights()##reset weight and bias
    
    #Loss Model
    criterion = nn.CrossEntropyLoss().to(device)
    #Optimizer Model
    optimizer = optim.SGD(net.parameters(), lr=LR)
    #Repeat
    nb_epoch=100000
    #Input data turn into Torch
    X = torch.FloatTensor(data_train)
    #print(X.shape) // [9999, 8]
    X = X.to(device)
    Y = torch.FloatTensor(data_y)
    #print(Y.shape) // [9999, 98]
    Y = Y.to(device)


    """#checking if Y is fine
    for i in range(len(data_y)):
        print(data_y[i].index(max(data_y[i])))
    """
    from tqdm import tqdm
    for epoch in tqdm(range(nb_epoch)):
        optimizer.zero_grad()
        # forward + backward + optimize
        outputs = net(X)
        loss = criterion(outputs, Y)
        loss.backward()
        optimizer.step()
    ##Evaluate
    print("At learning rate %.1E"%LR,end="  ")##print learning rate
    ## accruacy
    sum = 0
    for i in (range(len(Y))): # range : 9999
        if torch.argmax(outputs[i]) == torch.argmax(Y[i]):
            sum = sum + 1
    print("Accrucary : %.2f"%(sum/len(Y)*100),end="  ")
    ## avrage cost
    sum = 0
    for i in (range(len(Y))): # range : 9999
        sum = torch.argmax(outputs[i]) - torch.argmax(Y[i]) + sum
    print("Mean difference %.2f"%(sum.item()/len(Y)),end=" ")
    print("Loss : %.5f"%loss.item())


100%|██████████| 100000/100000 [01:45<00:00, 951.48it/s]


At learning rate 1.0E-03  Accrucary : 58.11  Mean difference -0.69 Loss : 1.50063


### Learning rate

At learning rate 1.0E+04  Accrucary : 49.85  Mean difference -18.04  
At learning rate 1.0E+02  Accrucary : 49.85  Mean difference -18.04  
At learning rate 1.0E+00  Accrucary : 49.85  Mean difference -18.04
##### At learning rate 1.0E-03  Accrucary : 56.82  Mean difference 0.01
At learning rate 1.0E-05  Accrucary : 50.19  Mean difference 2.05  
At learning rate 1.0E-10  Accrucary : 0.51  Mean difference 28.17  

## END

2023/01/24

0152i

it took about... 2 hours  
I am goin g to sleep now  